### Transformer Block 搭建

#### 1. 残差连接 
$$ y = x + f(x) $$

- 梯度消失：在反向传播时，残差路径是一条直达到底层的快速通道。梯度不需要再复杂的权重森林里迷路，它顺着内条先，直接回去  

- 网络退化：当网络深度增加到一定程度时，准确率会达到饱和甚至下降，这不是因为过拟合，而是深层网络难以学到“恒等变换”。残差连接通过$x+F(x)$的方式，将输入和输出进行“加法”，从而避免了网络退化

#### 2. Pre-Norm和Post-Norm
- Pre-Norm：原始transformer论文采用的方案，归一化放在残差连接之后。训练极不稳定，通常许哟复杂的“warm-up"策略来防止训练初期崩溃。结构为：$x_n+1=Norm(x_n)+F(x)$
- Post-Norm：归一化放在（Attention或FFN）之后，训练稳定，结构为：$x_n+1=F(Norm(x_n))$。由于归一化在残差路径之外，它保持了梯度传播的平滑，不需要复杂的预热策略也能稳定训练数百层的超深模型

#### 3.Block拆分为两个部分
##### 3.1 空间域通信（MHA）  
空间指的为位置.一段话有10个词,就有10个空间位置.由于Transformer并行计算."空间域通信"是让这些词相互连通  
$$ x=x+self.atten(self.lnl(x),tokenz_positions=token_positions) $$

- 流程:输入先经过ln1归一化,再进入atten
- 注意点:必须将token_positions传递给self.atten

#####  3.2 通道域提纯(FFN)

$$ x=x+self.ffn(self.ln2(x)) $$

- 流程:上一步的输出再次经过ln2归一化,进入ffn
- 注意点:FFN是逐位计算的,不依赖位置信息,因此不需要传位置参数

In [6]:
import torch
import torch.nn as nn 

In [7]:

class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(d_model, n_heads)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model)
        )
    
    def forward(self, x):
        # 注意力层 + 残差
        residual = x
        x = self.norm1(x)
        attn_out, _ = self.attn(x, x, x)
        x = residual + attn_out
        
        # 前馈网络 + 残差
        residual = x
        x = self.norm2(x)
        ffn_out = self.ffn(x)
        x = residual + ffn_out
        
        return x